In [1]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from scipy.sparse import rand as sprand
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils import data



In [2]:
# The grand dataset
ratings = pd.read_csv('ratings_small.csv', usecols = ['userId','movieId','rating'])
#ratings
#ratings.movieId


In [3]:
import torch
from torch.utils import data

# Formatting the data 

class MovieLens(Dataset):
  #'Characterizes a dataset for PyTorch'
    def __init__(self,csv_file,root_dir):
        'Initialization'
        self.movieLens = pd.read_csv(csv_file)
        self.root_dir = root_dir
        
    def __len__(self):
     #   'Denotes the total number of samples'
        return len(self.movieLens)

    def __getitem__(self, index):
     #   'Generates one sample of data'
        # Select sample
        
        movieId = self.movieLens.movieId.iloc[index]
        userId =self.movieLens.userId[index]
        rating = self.movieLens.rating[index]
        #obs = {'movieId':movieId,'userId':userId,'rating':rating}
        #obs = self.movieLens.drop('timestamp',axis=1)
        #obs = obs.iloc[index,:].as_matrix()

        return [movieId,userId,rating]
    

In [4]:



x = MovieLens(csv_file='ratings_small.csv',root_dir=None)
print(x[1])
print(len(x))

[1029, 1, 3.0]
100004


In [5]:
shuffle_dataset = True
train_split=0.7
validation_split=0.2
test_split=0.1
random_seed= 42
dataset_size = len(x)
indices = list(range(dataset_size))

               
split_1 = int(np.floor(train_split * dataset_size))
split_2 = split_1+int(np.floor(validation_split * dataset_size))
split_3 = split_2+int(np.floor(test_split * dataset_size))
split_2end=split_3-split_2

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices, test_indices = indices[:split_1], indices[split_1:-split_2end], indices[split_2:]

#train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(x, [train_size, test_size,val_size])
train_dataset, test_dataset, val_dataset = x[train_indices], x[val_indices], x[test_indices]



In [6]:
train_dataloader = DataLoader(train_dataset,batch_size = 4,shuffle=True)

#print(train_dataloader)

#for i_batch, sample_batched in enumerate(train_dataloader):
#    print(i_batch)
#    print(sample_batched)


In [7]:
test_dataloader = DataLoader(test_dataset,batch_size = 4,shuffle=True)


In [8]:
valid_dataloader = DataLoader(val_dataset,batch_size = 4,shuffle=True)


In [9]:
# instantiating model class
num_users = len(ratings.userId[train_indices].unique())
num_items = len(ratings.movieId[train_indices].unique())
print(num_users, num_items)

671 8010


In [10]:
#users = torch.LongTensor(ratings.userId.values)
#movies = torch.LongTensor(ratings.movieId.values)
#ratings = torch.FloatTensor(ratings.rating.values)


In [30]:
ratings.size

<function Tensor.size>

In [12]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        # init 
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self,userId,movieId):
        U = self.user_emb(userId)
        V = self.item_emb(movieId)
        b_u = self.user_bias(userId).squeeze()
        b_v = self.item_bias(movieId).squeeze()
        return (U*V).sum(1) +  b_userId  + b_movieId

In [14]:
model = MF_bias(num_users, num_items, emb_size=100)

NameError: name 'movieId' is not defined

In [ ]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    parameters = filter(lambda p: p.requires_grad, model.parameters()) # get all parameters which need grad
    optimizer = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    model.train() # into training mode
    for i in range(epochs):
        running_loss = 0.0
        
        for j, data in enumerate(train_loader):
            users, items, ratings = data
            users = Variable(users)
            items = Variable(items)
            ratings = Variable(ratings).float()
            
            users = users.cuda() # put on gpu
            items = items.cuda()
            ratings = ratings.cuda()            
            if unsqueeze:
                ratings = ratings.unsqueeze(1)
            y_hat = model(users, items)
            loss = F.mse_loss(y_hat, ratings)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.data[0]
        print("training loss for epoch ",i+1, ": ", running_loss/j+1) # used to be loss.data[0]
    test_loss(model, unsqueeze)


In [ ]:
###NEDESTÅENDE ER GAMMEL LORT ### 

In [ ]:

user=ratings.userId
user = user.values

item = ratings.movieId
item = item.values


#ratings = torch.tensor(ratings.values)
#ratings=ratings.toarray()
print(user)
print(item)
n_users = len(user)
n_items= len(item)


ratings = ratings.pivot(index='userId', columns='movieId', values='rating')
mat = ratings.values
mat[np.isnan(mat)] = 0
#mat=mat.astype(int)
print(mat)

In [ ]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=True)
        #print(self.user_factors)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=True)
        
    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)

In [ ]:
model = MatrixFactorization(n_users, n_items, n_factors=20)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=1e-6) # learning rate

In [ ]:
rows, cols = mat.nonzero()
print(rows)
print(cols)
p = np.random.permutation(len(rows))
print(p)
rows, cols = rows[p], cols[p]
print(rows)
print(cols)

for row, col in zip(*(rows, cols)):
    # Turn data into variables
    rating = Variable(torch.FloatTensor([mat[row, col]]))
    row = Variable(torch.LongTensor([np.long(row)]))
    col = Variable(torch.LongTensor([np.long(col)]))
    
    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)
    
    # Backpropagate
    loss.backward()
    
    # Update the parameters
    optimizer.step()
    
    print(rating)
    print(row)
    print(col)
    print(prediction)
    print(loss)
  